In [1]:
import os, pandas as pd

BASE    = "/ShangGaoAIProjects/Lingge/LINCS/data"
PROC    = f"{BASE}/Processed_data"
OUT_DIR = f"{PROC}/L1000gctx_process"

CAT_PATH = f"{OUT_DIR}/Y_landmark_train_catalog_canonical.index.parquet"
CAT = pd.read_parquet(CAT_PATH)

print("Catalog shape:", CAT.shape)
display(CAT.head(3))

# 我们将使用这几列（如缺失就提醒你）
need_cols = ["sig_id", "smiles_canonical", "cell_id", "dose_uM"]
missing = [c for c in need_cols if c not in CAT.columns]
if missing:
    print("⚠️ 缺少列：", missing)


Catalog shape: (209540, 18)


,sig_id,phase,pert_id,pert_iname,cell_id,time_h,dose_uM,dose_value,dose_unit_raw,smiles,smiles_canonical,inchi_key,compound_id,is_small_molecule,is_control,pert_type,col_pos,dataset
0,AML001_CD34_24H:BRD-A03772856:0.37037,GSE92742,BRD-A03772856,BRD-A03772856,cd34,24.0,0.37037,0.37037,µM,COc1ccccc1C2N(C(=O)C3CCCN23)c4ccc(Cl)cc4,COc1ccccc1C1N(c2ccc(Cl)cc2)C(=O)C2CCCN21,YSPMFQJSWDVJME-UHFFFAOYSA-N,YSPMFQJSWDVJME-UHFFFAOYSA-N,True,False,trt_cp,434316,GSE92742
1,AML001_CD34_24H:BRD-A03772856:1.11111,GSE92742,BRD-A03772856,BRD-A03772856,cd34,24.0,1.11111,1.11111,µM,COc1ccccc1C2N(C(=O)C3CCCN23)c4ccc(Cl)cc4,COc1ccccc1C1N(c2ccc(Cl)cc2)C(=O)C2CCCN21,YSPMFQJSWDVJME-UHFFFAOYSA-N,YSPMFQJSWDVJME-UHFFFAOYSA-N,True,False,trt_cp,434315,GSE92742
2,AML001_CD34_24H:BRD-A03772856:10,GSE92742,BRD-A03772856,BRD-A03772856,cd34,24.0,10.00000,10.00000,µM,COc1ccccc1C2N(C(=O)C3CCCN23)c4ccc(Cl)cc4,COc1ccccc1C1N(c2ccc(Cl)cc2)C(=O)C2CCCN21,YSPMFQJSWDVJME-UHFFFAOYSA-N,YSPMFQJSWDVJME-UHFFFAOYSA-N,True,False,trt_cp,434313,GSE92742


In [8]:
# STEP 16: 从 Hugging Face 下载 MoLFormer 并对唯一 SMILES 编码（FP32，避免 NaN/Inf）
import os, numpy as np, pandas as pd, torch
from transformers import AutoTokenizer, AutoModel

# ====== 路径与输入 ======
BASE     = "/ShangGaoAIProjects/Lingge/LINCS/data"
PROC     = f"{BASE}/Processed_data"
OUT_DIR  = f"{PROC}/L1000gctx_process"
CAT_PATH = f"{OUT_DIR}/Y_landmark_train_catalog_canonical.index.parquet"

# 选用的公开权重（MoLFormer）
HF_MODEL_ID = "ibm-research/MoLFormer-XL-both-10pct"
# 可选：指定 HF 本地缓存目录（避免每次重复下载）
HF_CACHE_DIR = f"{BASE}/Models/HF_cache"
os.makedirs(HF_CACHE_DIR, exist_ok=True)

# ====== 读取 catalog，提取唯一 SMILES ======
CAT = pd.read_parquet(CAT_PATH)
SMI_COL = "smiles_canonical" if "smiles_canonical" in CAT.columns else "smiles"
assert SMI_COL in CAT.columns, f"catalog 缺少 {SMI_COL} 列"

smiles_series = CAT[SMI_COL].astype(str)
mask_valid = (~smiles_series.isna()) & (smiles_series.str.len() > 0) & (smiles_series != "-666")
uniq_smiles = smiles_series[mask_valid].drop_duplicates().tolist()
print("Unique valid SMILES:", len(uniq_smiles))

# ====== 设备与模型（下载自 HF，信任自定义代码；强制 FP32）======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

tokenizer = AutoTokenizer.from_pretrained(
    HF_MODEL_ID,
    trust_remote_code=True,
    cache_dir=HF_CACHE_DIR
)
model = AutoModel.from_pretrained(
    HF_MODEL_ID,
    trust_remote_code=True,
    cache_dir=HF_CACHE_DIR
)

# 某些 tokenizer 可能没有 pad_token，给个兜底
if tokenizer.pad_token is None:
    # 优先用 eos，其次添加一个新 pad token
    if tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token
    else:
        tokenizer.add_special_tokens({"pad_token": "[PAD]"})
        model.resize_token_embeddings(len(tokenizer))

model.eval().to(device).to(torch.float32)   # 关键：FP32，避免 NaN/Inf
torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = False

# ====== 编码（批量；禁用 autocast；每批做有限值检查）======
batch_size = 256  # 显存不够就降到 128/64
embeds = []
bad_spans = 0

def forward_encode(smiles_list, dev):
    toks = tokenizer(
        smiles_list,
        padding=True,
        truncation=True,
        return_tensors="pt",
    ).to(dev)
    out = model(**toks)  # 不用 autocast
    if hasattr(out, "pooler_output") and out.pooler_output is not None:
        emb = out.pooler_output
    elif hasattr(out, "last_hidden_state"):
        emb = out.last_hidden_state[:, 0, :]  # CLS
    elif isinstance(out, (tuple, list)) and len(out) > 0:
        emb = out[0][:, 0, :]
    else:
        raise RuntimeError("无法从 MoLFormer 输出结构中找到 embedding 向量。")
    return emb.detach().cpu().numpy().astype(np.float32)

for i in range(0, len(uniq_smiles), batch_size):
    chunk = uniq_smiles[i:i+batch_size]
    # 首选：device（GPU/CPU）FP32
    try:
        emb_np = forward_encode(chunk, device)
    except Exception as e:
        print(f"[forward error @ {i}:{i+batch_size} on {device}] {e}")
        emb_np = None

    # 有限值检查；如失败，回退到 CPU 再算一次
    if emb_np is None or not np.isfinite(emb_np).all():
        try:
            emb_np_cpu = forward_encode(chunk, torch.device("cpu"))
            emb_np = emb_np_cpu
        except Exception as e2:
            print(f"[fallback CPU error @ {i}:{i+batch_size}] {e2}")
            # 兜底：本批置零，后续你也可以改成跳过
            d_tmp = emb_np.shape[1] if isinstance(emb_np, np.ndarray) and emb_np.ndim == 2 else 768
            emb_np = np.zeros((len(chunk), d_tmp), dtype=np.float32)
            bad_spans += 1

    # 最终确保没有 NaN/Inf（若仍有，置零）
    if not np.isfinite(emb_np).all():
        mask_bad = ~np.isfinite(emb_np).all(axis=1)
        if mask_bad.any():
            emb_np[mask_bad] = 0.0

    embeds.append(emb_np)

emb_matrix = np.vstack(embeds).astype(np.float32)
d_emb = emb_matrix.shape[1]
finite_ratio = float(np.isfinite(emb_matrix).mean())
print("Embedding matrix:", emb_matrix.shape, "| finite ratio:", finite_ratio, "| bad batches:", bad_spans)

# ====== 保存唯一库（SMILES→embedding 行号）======
uniq_df = pd.DataFrame({SMI_COL: uniq_smiles})
uniq_df["emb_row"] = np.arange(len(uniq_smiles), dtype=np.int32)

uniq_smiles_path = f"{OUT_DIR}/uniq_smiles_with_emb_row.parquet"
np.save(f"{OUT_DIR}/molformer_unique_embeddings.npy", emb_matrix)
uniq_df.to_parquet(uniq_smiles_path, index=False)

print("Saved:")
print(" ", uniq_smiles_path)
print(" ", f"{OUT_DIR}/molformer_unique_embeddings.npy")


Unique valid SMILES: 9866
Using device: cuda


tokenizer_config.json: 1.29kB [00:00, 3.28MB/s]
tokenization_molformer_fast.py: 6.50kB [00:00, 18.9MB/s]
A new version of the following files was downloaded from https://huggingface.co/ibm-research/MoLFormer-XL-both-10pct:
- tokenization_molformer_fast.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
vocab.json: 41.6kB [00:00, 86.9MB/s]
tokenizer.json: 54.0kB [00:00, 105MB/s]
special_tokens_map.json: 100%|█████████████████████████████████████████████████████████| 125/125 [00:00<00:00, 1.05MB/s]
config.json: 1.01kB [00:00, 3.30MB/s]
configuration_molformer.py: 7.60kB [00:00, 23.7MB/s]
A new version of the following files was downloaded from https://huggingface.co/ibm-research/MoLFormer-XL-both-10pct:
- configuration_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
modeling

Embedding matrix: (9866, 768) | finite ratio: 1.0 | bad batches: 0
Saved:
  /ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/L1000gctx_process/uniq_smiles_with_emb_row.parquet
  /ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/L1000gctx_process/molformer_unique_embeddings.npy


In [1]:
import os, json
import numpy as np
import pandas as pd

OUT_DIR = "/ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/L1000gctx_process"

# 载入在 Step 16 保存的唯一 SMILES 与向量库
uniq_df    = pd.read_parquet(f"{OUT_DIR}/uniq_smiles_with_emb_row.parquet")  # 包含列：smiles_canonical / 或 smiles、emb_row
emb_matrix = np.load(f"{OUT_DIR}/molformer_unique_embeddings.npy")           # 形状 (n_unique, d)

n, d = emb_matrix.shape
print(f"Embedding matrix shape: {n} × {d}")

# 1) 基础数值检查：NaN / Inf / 行范数
has_nan = np.isnan(emb_matrix)
has_inf = ~np.isfinite(emb_matrix)

row_nan = has_nan.any(axis=1)
row_inf = has_inf.any(axis=1)
row_norm = np.linalg.norm(emb_matrix, axis=1)

n_nan = int(row_nan.sum())
n_inf = int(row_inf.sum())
n_zero = int((row_norm < 1e-8).sum())

print(f"Rows with NaN: {n_nan}")
print(f"Rows with Inf:  {n_inf}")
print(f"Zero-norm rows: {n_zero}")

# 2) 范数分布摘要（看有没有特别离谱的向量）
norm_stats = {
    "mean": float(row_norm.mean()),
    "std":  float(row_norm.std()),
    "min":  float(row_norm.min()),
    "p1":   float(np.percentile(row_norm, 1)),
    "p5":   float(np.percentile(row_norm, 5)),
    "p50":  float(np.percentile(row_norm, 50)),
    "p95":  float(np.percentile(row_norm, 95)),
    "p99":  float(np.percentile(row_norm, 99)),
    "max":  float(row_norm.max()),
}
print("Norm stats:", norm_stats)

# 3) 维度层面分布（极端维度有时提示编码异常）
dim_mean = emb_matrix.mean(axis=0)
dim_std  = emb_matrix.std(axis=0)
dim_std_small = int((dim_std < 1e-6).sum())  # 基本不变的维度数
print(f"Dims with ~zero std: {dim_std_small} / {d}")

# 4) 精确重复向量检测（逐行完全一样）
#    用“按行唯一”方式找重复：把 float32 转成 bytes view 来做 exact match
#    注：这是“完全一致”的严格重复，近似重复不在此列
view = emb_matrix.view([('f', emb_matrix.dtype, d)])  # 把每行视作一个复合类型
_, unique_idx, inverse = np.unique(view, return_index=True, return_inverse=True)
n_unique_rows = len(unique_idx)
n_dupe_rows = n - n_unique_rows
print(f"Exact duplicate embeddings: {n_dupe_rows}")

# 5) 判定异常的规则与导出
#    - 含 NaN/Inf
#    - 零范数
#    - 范数离群：使用 IQR（Q1-3*IQR, Q3+3*IQR）或 > mean+6*std 作为保守阈值
q1, q3 = np.percentile(row_norm, [25, 75])
iqr = q3 - q1
low_thr = q1 - 3 * iqr
high_thr = q3 + 3 * iqr
six_sigma = (row_norm > norm_stats["mean"] + 6*norm_stats["std"]) | (row_norm < max(1e-12, norm_stats["mean"] - 6*norm_stats["std"]))

is_outlier_norm = (row_norm < low_thr) | (row_norm > high_thr) | six_sigma

# 整理 DataFrame
smiles_col = "smiles_canonical" if "smiles_canonical" in uniq_df.columns else "smiles"
qc_df = pd.DataFrame({
    "emb_row": uniq_df["emb_row"].to_numpy(),
    smiles_col: uniq_df[smiles_col].astype(str).to_numpy(),
    "has_nan": row_nan,
    "has_inf": row_inf,
    "norm": row_norm,
    "outlier_norm": is_outlier_norm,
})

# 标记精确重复（除第一条之外的重复行）
# inverse 给出每行对应的“唯一代表”的编号；相同编号表示重复
# 统计每个代表的出现次数，>1 的都属于重复簇
rep_counts = np.bincount(inverse)
is_dupe = rep_counts[inverse] > 1
qc_df["exact_duplicate_vec"] = is_dupe

# 汇总异常数量
summary = {
    "n_rows": n,
    "n_dims": d,
    "n_nan_rows": int(qc_df["has_nan"].sum()),
    "n_inf_rows": int(qc_df["has_inf"].sum()),
    "n_zero_norm_rows": int((qc_df["norm"] < 1e-8).sum()),
    "n_outlier_norm_rows": int(qc_df["outlier_norm"].sum()),
    "n_exact_duplicate_rows": int(qc_df["exact_duplicate_vec"].sum()),
    "norm_stats": norm_stats,
    "iqr_bounds": {"low": float(low_thr), "high": float(high_thr)},
}

print("QC summary:", summary)

Embedding matrix shape: 9866 × 768
Rows with NaN: 0
Rows with Inf:  0
Zero-norm rows: 0
Norm stats: {'mean': 16.07752227783203, 'std': 0.9821127653121948, 'min': 13.601053237915039, 'p1': 14.305994558334351, 'p5': 14.756412982940674, 'p50': 15.933931350708008, 'p95': 17.908660888671875, 'p99': 18.93819389343262, 'max': 24.17510414123535}
Dims with ~zero std: 0 / 768
Exact duplicate embeddings: 0
QC summary: {'n_rows': 9866, 'n_dims': 768, 'n_nan_rows': 0, 'n_inf_rows': 0, 'n_zero_norm_rows': 0, 'n_outlier_norm_rows': 25, 'n_exact_duplicate_rows': 0, 'norm_stats': {'mean': 16.07752227783203, 'std': 0.9821127653121948, 'min': 13.601053237915039, 'p1': 14.305994558334351, 'p5': 14.756412982940674, 'p50': 15.933931350708008, 'p95': 17.908660888671875, 'p99': 18.93819389343262, 'max': 24.17510414123535}, 'iqr_bounds': {'low': 11.833098411560059, 'high': 20.168317556381226}}


In [2]:
import numpy as np, pandas as pd, os

OUT_DIR  = "/ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/L1000gctx_process"
CAT_PATH = f"{OUT_DIR}/Y_landmark_train_catalog_canonical.index.parquet"

CAT = pd.read_parquet(CAT_PATH)
SMI_COL = "smiles_canonical" if "smiles_canonical" in CAT.columns else "smiles"

uniq_df    = pd.read_parquet(f"{OUT_DIR}/uniq_smiles_with_emb_row.parquet")  # SMI_COL, emb_row
emb_matrix = np.load(f"{OUT_DIR}/molformer_unique_embeddings.npy")           # (n_unique, 768)
d_emb = emb_matrix.shape[1]

# 构建映射：SMILES → 行号
smiles_to_row = dict(zip(uniq_df[SMI_COL].tolist(), uniq_df["emb_row"].tolist()))
emb_row_idx = CAT[SMI_COL].astype(str).map(smiles_to_row)

# 理论上不应有缺失；若有，给个显式报错以免静默错配
if emb_row_idx.isna().any():
    miss_n = int(emb_row_idx.isna().sum())
    print("⚠️ 有未命中的 SMILES 行数：", miss_n)
    print(CAT.loc[emb_row_idx.isna(), [SMI_COL]].head())
    raise ValueError("存在 SMILES 未命中唯一库，请先排查。")

emb_row_idx = emb_row_idx.to_numpy(dtype=np.int64)

# 写 raw 版本
N = len(CAT)
X_raw_path = f"{OUT_DIR}/X_mol_molformer_{d_emb}d.npy"
X_raw = np.memmap(X_raw_path, mode="w+", dtype="float32", shape=(N, d_emb))
print(X_raw.shape)

stride = 50000
for i in range(0, N, stride):
    sl = slice(i, min(i+stride, N))
    rows = emb_row_idx[sl]
    X_raw[sl, :] = emb_matrix[rows, :]

del X_raw  # 关闭写句柄
print("Saved RAW mol embeddings:", X_raw_path)

# 同步写 L2-normalized 版本（逐行）
X_l2_path = f"{OUT_DIR}/X_mol_molformer_{d_emb}d_l2.npy"
X_l2 = np.memmap(X_l2_path, mode="w+", dtype="float32", shape=(N, d_emb))
print(X_l2.shape)

# 为节省内存，分块做 L2
def _l2norm(a, axis=1, eps=1e-8):
    n = np.linalg.norm(a, axis=axis, keepdims=True)
    n[n < eps] = 1.0
    return a / n

emb_mem = np.memmap(X_raw_path, mode="r", dtype="float32", shape=(N, d_emb))
for i in range(0, N, stride):
    sl = slice(i, min(i+stride, N))
    X_l2[sl, :] = _l2norm(emb_mem[sl, :])

del X_l2, emb_mem
print("Saved L2-normalized mol embeddings:", X_l2_path)


(209540, 768)
Saved RAW mol embeddings: /ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/L1000gctx_process/X_mol_molformer_768d.npy
(209540, 768)
Saved L2-normalized mol embeddings: /ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/L1000gctx_process/X_mol_molformer_768d_l2.npy
